In [28]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers


data = pd.read_csv("data.csv", sep=";" )

print(data.columns)

Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month'],
      dtype='object')


In [29]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

omit = []
X = data.drop('default payment next month', axis=1)
X = X.drop(omit, axis=1)
y = data['default payment next month']

def deleteFromList(list, omition_list):
    return [item for item in list if item not in omition_list]




# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_data = pd.concat([X_train, y_train], axis=1)

# Separa las clases
non_default = train_data[train_data['default payment next month'] == 0]
default = train_data[train_data['default payment next month'] == 1]

# Realiza el submuestreo (undersampling)
non_default_undersampled = non_default.sample(len(default), random_state=42)

# Combina las clases de nuevo
undersampled_data = pd.concat([non_default_undersampled, default])

# Separa las características y las etiquetas de nuevo
X_train_undersampled = undersampled_data.drop('default payment next month', axis=1)
y_train_undersampled = undersampled_data['default payment next month']


#X_train = X_train_undersampled
#y_train = y_train_undersampled

# Columnas numéricas y categóricas
num_attribs = deleteFromList(['LIMIT_BAL', 'AGE', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6'], omit)
cat_attribs = deleteFromList(['SEX', 'EDUCATION', 'MARRIAGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6'], omit)

# Tubos de preprocesamiento
num_pipeline = Pipeline([
        ('std_scaler', StandardScaler()),
    ])

cat_pipeline = Pipeline([
        ('encoder', OneHotEncoder()),
    ])

# Combinar todo
preprocessor = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
    ])

X_train_prepared = preprocessor.fit_transform(X_train)
X_test_prepared = preprocessor.transform(X_test)


In [30]:
# %%
# Construir el modelo
from tensorflow.keras import layers, regularizers


model = keras.Sequential([
 # Añadir Dropout después de la primera capa oculta con una tasa del 20%
    layers.Dense(64, activation='relu' ),
    layers.Dense(32, activation='relu' ),
    layers.Dense(16, activation='relu'),

    layers.Dense(8, activation='relu'),

    layers.Dense(1, activation='sigmoid')
])

# Compilar el modelo
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Compilar el modelo
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [31]:
X_train_full, X_val, y_train_full, y_val = train_test_split(
    X_train_prepared, y_train, test_size=0.2, random_state=42)

X_train_full = X_train_full.toarray()
X_val = X_val.toarray()




history = model.fit(
    X_train_full, y_train_full,
    epochs=50,
    batch_size=100,
    validation_data=(X_val, y_val)
)


Epoch 1/40
1200/1200 [==============================] - 3s 2ms/step - loss: 0.4435 - accuracy: 0.8154 - val_loss: 0.4377 - val_accuracy: 0.8152
Epoch 2/40
1200/1200 [==============================] - 2s 2ms/step - loss: 0.4291 - accuracy: 0.8218 - val_loss: 0.4370 - val_accuracy: 0.8148
Epoch 3/40
1200/1200 [==============================] - 2s 2ms/step - loss: 0.4256 - accuracy: 0.8225 - val_loss: 0.4392 - val_accuracy: 0.8152
Epoch 4/40
1200/1200 [==============================] - 2s 2ms/step - loss: 0.4234 - accuracy: 0.8243 - val_loss: 0.4397 - val_accuracy: 0.8154
Epoch 5/40
1200/1200 [==============================] - 2s 2ms/step - loss: 0.4201 - accuracy: 0.8248 - val_loss: 0.4363 - val_accuracy: 0.8123
Epoch 6/40
1200/1200 [==============================] - 2s 2ms/step - loss: 0.4182 - accuracy: 0.8255 - val_loss: 0.4398 - val_accuracy: 0.8142
Epoch 7/40
1200/1200 [==============================] - 2s 2ms/step - loss: 0.4150 - accuracy: 0.8269 - val_loss: 0.4380 - val_accuracy:

In [32]:
model.evaluate(X_test_prepared.toarray(), y_test, verbose = 2)

188/188 - 0s - loss: 0.5627 - accuracy: 0.8037 - 356ms/epoch - 2ms/step


[0.5627308487892151, 0.8036666512489319]